In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook
from openpyxl.drawing.image import Image

In [2]:
data = {
    "StudentID": [101, 102, 103, 104, 105, 106, 107],
    "Name": ["Alice", "Bob", "Charlie", "David", "Emma", "Frank", "Grace"],
    "Math": [95, 72, 88, 55, 80, 99, 68],
    "Physics": [89, 65, 91, 62, 77, 95, 72],
    "Chemistry": [92, 70, 85, 58, 79, 97, 74],
    "Biology": [88, 60, 90, 61, 83, 96, 70]
}

df = pd.DataFrame(data)

In [3]:
df["Total"] = df[["Math", "Physics", "Chemistry", "Biology"]].sum(axis=1)
df["Average"] = df[["Math", "Physics", "Chemistry", "Biology"]].mean(axis=1)

In [4]:
conditions = [
    (df["Average"] >= 90),
    (df["Average"] >= 75) & (df["Average"] < 90),
    (df["Average"] >= 60) & (df["Average"] < 75),
    (df["Average"] < 60)
]
grades = ["A", "B", "C", "F"]
df["Grade"] = np.select(conditions, grades, default="F")

In [5]:
top_performers = {}
for subject in ["Math", "Physics", "Chemistry", "Biology"]:
    top3 = df.nlargest(3, subject)[["StudentID", "Name", subject]]
    top_performers[subject] = top3

In [6]:
top_performers_df = pd.concat(
    top_performers, names=["Subject"]
).reset_index(level=0)

In [7]:
avg_marks = df[["Math", "Physics", "Chemistry", "Biology"]].mean()

plt.figure(figsize=(8, 5))
avg_marks.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Average Marks per Subject")
plt.ylabel("Average Marks")
plt.tight_layout()
plt.savefig("average_marks_chart.png")
plt.close()

In [8]:
with pd.ExcelWriter("results.xlsx", engine="openpyxl") as writer:
    df[["StudentID", "Name", "Total", "Average", "Grade"]].to_excel(
        writer, sheet_name="Summary", index=False
    )
    top_performers_df.to_excel(writer, sheet_name="Top Performers", index=False)
    writer.book.save("results.xlsx")

In [9]:
wb = load_workbook("results.xlsx")
ws = wb.create_sheet("Chart")
img = Image("average_marks_chart.png")
ws.add_image(img, "A1")
wb.save("results.xlsx")
print(" Excel file 'results.xlsx' created successfully with Summary, Top Performers, and Chart.")

 Excel file 'results.xlsx' created successfully with Summary, Top Performers, and Chart.
